In [2]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("flax-community/t5-recipe-generation")

model = TFAutoModelForSeq2SeqLM.from_pretrained("flax-community/t5-recipe-generation")

/home/kay_r0hit/Rohit/sem2/NLP/chef-bot/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-14 14:11:33.537489: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-14 14:11:34.524439: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-14 14:11:34.524543: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not 

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("../dataset/IndianFoodDatasetCSV.csv")

In [5]:
# df2 = pd.read_csv('../dataset/test.csv')

In [6]:
new_df = df[['TranslatedIngredients','TranslatedInstructions']]

In [7]:
new_df.head()

,TranslatedIngredients,TranslatedInstructions
0,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...","To begin making the Masala Karela Recipe,de-se..."
1,"2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...","To make tomato puliogere, first cut the tomato..."
2,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...","To begin making the Ragi Vermicelli Recipe, fi..."
3,"500 grams Chicken,2 Onion - chopped,1 Tomato -...",To begin making Gongura Chicken Curry Recipe f...
4,"1 tablespoon chana dal, 1 tablespoon white ura...","To make Andhra Style Alam Pachadi, first heat ..."


In [19]:
new_df.shape

(6871, 2)

In [14]:
def get_tokenized(text):
    return tokenizer(str(text), padding=True, truncation=True, return_attention_mask=True,add_special_tokens=True)

input_embeddings = new_df['TranslatedIngredients'].map(get_tokenized)
output_embeddings = new_df['TranslatedInstructions'].map(get_tokenized)
# input_embeddings = tokenizer(input_data, padding=True, truncation=True, return_attention_mask=True,add_special_tokens=True)

In [22]:
prefix = "items: "
text_column = "inputs"
target_column = "targets"
max_source_length = 256
max_target_length = 1024
seed = 42
eval_batch_size = 64

In [23]:
generation_kwargs = {
    "max_length": 1024,
    "min_length": 64,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
    "num_beams": 4,
    "length_penalty": 1.5,
}

In [24]:
special_tokens = tokenizer.all_special_tokens
tokens_map = {
    "<sep>": "--",
    "<section>": "\n"
}

In [25]:
def skip_special_tokens(text, special_tokens):
    for token in special_tokens:
        text = text.replace(token, '')

    return text

def target_postprocessing(texts, special_tokens):
    if not isinstance(texts, list):
        texts = [texts]
    
    new_texts = []
    for text in texts:
        text = skip_special_tokens(text, special_tokens)

        for k, v in tokens_map.items():
            text = text.replace(k, v)

        new_texts.append(text)

    return new_texts